<table style="background-color:#F5F5F5;" width="100%">
<tr><td style="background-color:#F5F5F5;"><img src="../images/logo.png" width="100" align='right'/></td></tr>     <tr><td>
            <h1><center>Aprendizagem Automática em Engenharia Biomédica</center></h1>
            <h3><center>1st Semester - 2024/2025</center></h3>
            <h4><center>Universidade Nova de Lisboa - Faculdade de Ciências e Tecnologia</center></h4>

</td></tr>
    <tr><td><h2><b><center>Lab 8 - Statistical Machine Learning</center></b></h2>
    <h4><i><b><center>Bayes Rule and Naive Bayes Classification</center></b></i></h4></td></tr>
</table>

## 1. Introduction to Statistical Machine Learning
Statistical machine learning is an approach that __grounds machine learning in probability and statistics__, allowing us to build models that can make reliable predictions based on uncertain or incomplete data. By modeling the underlying probability distributions of data, statistical methods capture the variability and randomness often present in real-world datasets, making them particularly effective for fields like biomedical engineering. This approach not only provides a mathematical framework for making predictions but also enables us to quantify uncertainty, interpret model decisions, and better understand the structure of complex data.

### 1.1 Discriminative vs. Generative Models
In statistical machine learning, models can generally be categorized into __discriminative__ and __generative__ types, each with different approaches to understanding the relationship between data and labels.

__1. Discriminative Models__: These models focus on learning the direct relationship between input features $X$ and their associated labels $Y$. Discriminative models estimate the __conditional probability__ $P(Y|X)$ (i.e., the probability of $Y$ given $X$),  which allows them to predict the label $Y$ for a given set of features $X$. By concentrating on the boundary between classes, discriminative models are often efficient and accurate in classification tasks. Common examples include, among others:
* Logisitic Regression
* Support Vector Machines (SVMs)
* Decision Trees
* Neural Networks
      
__2. Generative Models__: On the other hand, generative models aim to model the __joint probability distribution__ $P(Y,X)$, which describes how the data and labels are jointly distributed. By learning this distribution, generative models can use __Bayes' Rule__ to derive $P(Y|X)$, enabling classification. This approach allows generative models to capture a fuller picture of the data, which can make them more versatile in tasks beyond classification, such as generating new data samples. Common examples of generative models include, among others:
* Naive Bayes
* Gaussian Mixture Models (HMMs)
* Hidden Markov Models (HMMs)

<div style="text-align:center;">
    <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*mi4d8qNEA5owx-EVm9VR1w.png" width="600">
</div>

### 1.2 Goal of Today's Class
In today’s class, we will delve into the __Naive Bayes classifier__, a generative model that uses Bayes' Rule with the simplifying assumption that all features are conditionally independent given the label. This "naive" assumption makes it a simple yet effective classifier, especially in cases where the independence assumption holds approximately true. We will do this by:
* Learn about Baye's Rule
* Implement the Naive Bayes classifier using scikit-learn

In [ ]:
# -- imports needed for the Notebook -- #

# %matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

from typing import Union

## 2. Foundations of Probability Theory

To understand statistical machine learning, it's essential to have a firm grasp of probability theory. In this section, we’ll cover some foundational probability concepts, including conditional probability, joint probability, and marginal probability. These concepts form the basis for Bayes' Rule, which is central to the Naive Bayes classifier.
 
__1. Marginal Probability__: the marginal probability of an event is the probability of that __event occurring independently of other events__. The notation is $P(A)$, for the probabilty of event $A$ happening. For example, if 40 out of 100 patients have a fever, then
$$P(Fever) =\frac{40}{100} = 0.4 $$

__2. Joint Probability__: The joint probability $P(A,B)$ is the probability of two events $A$ and $B$ both occuring simultaneously. For example, $P(Fever, Cough)$ might represent the probability that a patient has __both__ fever and cough.

__3. Conditional Probability__: The coditional probability $P(A|B)$ represents the probabilty of event $A$ occuring given that $B$ __has already occured__. For example $P(Fever|Cough)$ is the probability of a patient __having a fever given that they already have a cough__. Conditional probability is a key component of Bayes' Rule.


### 3 Bayes' Rule
Bayes' Rule is a fundamental theorem in probability theory that allows us to update our beliefs about the world in light of new data. It provides a framework for computing the __posterior probability__ of a hypothesis $Y$ (such as a class label) given observed data $X$, by relating it to the __prior probability__ and the __likelihood__ of the data.

Mathematically, Bayes' Rule is expressed as:

$$ P(Y|X) = \frac{P(X|Y)P(Y)}{P(X)}$$

Where:
* $P(Y|X)$ __posterior probability__: the probability of the label $Y$ given the observed data $X$.
* $P(X|Y)$ __likelihood__: the probability observing the data $X$ given the label $Y$. This term is often referred to as the likelihood because, in statistical modeling, it expresses how likely the observed data is under a particular hypothesis or model.
* $P(Y)$ __prior probability__: the initial probability of the label Y before observing the data. This reflects what we know about the data (i.e., what is the occurence of that label within the dataset) before incorporating the data.
* $P(X)$ __marginal probability (or evidence)__: the __total probability__ of observing the data $X$ across all possible labels. It normalizes the equation so that the posterior probability sums to 1.

_Note_: In this notation $X$ does not stand for our entire feature matrix. In this case $X$ stands for a single feature.

#### 3.1 A Simple Example
While all of this sounds relatively abstract, we can easily think of Bayes' Rule as geometrical proprotions of a square. But before we think about squares, let's calculate an example using the forumula above:

We have a total population of __100__ people. Of these 100 people
* 10 % of the people have a respiratory disease. Thus, $$P(Y=Disease)=\frac{10}{100}=0.1$$
* 80 % of the people having the respiratory disease have a cough as symptom. Thus, $$P(X=Cough|Y=Disease)=\frac{8}{10}=0.8$$
* 30 % percent of the __entire population__, regardless of whether they have the disease or not, have a cough. Thus, $$P(X=Cough)=\frac{30}{100}=0.3$$

Therefore, $$P(Y=Disease|X=Cough)=\frac{P(X=Cough|Y=Disease)\cdot P(Y=Disease)}{P(X=Cough)}=\frac{0.8 \cdot 0.1}{0.3}=0.2\bar{6}$$

Looking at the formula now, we can give clear an deasy definitions for the __numerator__ (the part above the fraction line) and the __denominator__ (the part below the fraction line):
* __numerator__: The proportion of people, from the entire population, that __have the disease and have a cough__.
* __denominator__: The proportion of people from the __entire popuplation that have a cough__ (regardless of having the disease or not).

Now that we understood this, let's visualize the above problem using a square of size 10x10 = 100, where each sub-square represents a person in the population. The people of the population that have the diseases are outlined in red, while the healthy people are outlined in green. The people having a cough are filled with a lighter shade of the respective color. This visualization was inspired by the YouTuber [__3Blue1Brown__](https://www.youtube.com/@3blue1brown) who has a great [__video on Bayes' Theorem__](https://www.youtube.com/watch?v=HZGCoVF3YvM).

_Note_: The image below uses $N(Disease)$, $N(Cough|Disease)$, and $N(Cough)$, where the $N$ stands for "Number of people", as opposed to the probabilities. This is done as porportions are easier understood using the actual numbers than the respective probabilities.

<div style="text-align:center;">
    <img src="https://raw.githubusercontent.com/hgamboa/nova-aaeb/refs/heads/main/images/Bayes%20Theorem.png" width="900">
</div>

#### 3.2 From Discrete Variables to Continuous Variables

In the previous sections, we dealt with discrete features, where we can calculate probabilities by simply counting occurrences. For example, the probability of a person having a cough (or not) was calculated based on a simple count across samples.

However, in real-world applications, many features are __continuous__ (e.g., temperature, heart rate, blood pressure). For continuous features, the probability of a single specific value, like  $x=37.2ºC$, is effectively zero because continuous variables have infinitely many possible values. Instead, we need to think in terms of __probability density functions (pdfs)__ rather than probabilities.

#### 3.2.1 Probability Density Functions (PDFs)
For continuous variables, we use a probability density function to describe the probability of observing values within a specific range. The pdf, typically denoted $p(x)$, allows llows us to calculate probabilities over intervals. For example, the probability of a temperature $X$ falling between 36 ºC and 38 ºC can be calculated by integrating the pdf in that interval:
$$P(36<X<38) = \int_{36}^{38}p(x) dx$$

where $p(x)$ is the pdf of $X$. Common pdfs include __Guassian distribution__ (normal distribution), the __exponential distribution__, and others.

#### 3.2.2 Bayes' Rule with Continuous Variables
Given the usage of pdfs, Bayes' Rule thus becomes:

$$ P(Y=y_i|X) = \frac{p(X|Y=y_i)P(Y=y_i)}{P(X)}$$

where:
* $P(Y=y_i|X)$ __posterior probability__: the probability of the label $Y$ given the observed data $X$.
* $p(X|Y=y_i)$ __likelihood__: the pdf of the feature vector X given that it belongs to class $y_i$.
* $P(Y=y_i)$ __prior probability__ of class $y_i$
* $P(X)$ __marginal likelihood (or evidence)__: found by integrating the likelihoods accross all classes:
$$p(X) = \sum_{j=1}^{C}p(X|Y=y_i)P(Y=y_i)$$ with $C$ being the total number of classes.

#### 3.3 Exercises
Let us consider a classification problem with two classes, $Y=y_1$ and $Y=y_2$, representing two different medical conditions. We are given one feature vector $X$, which could be soemething like a biomarker measurement or a symptom score. We further observe that the distribution of the feature for each class follows a __normal distribution__ (or gaussian), which is defined by

<span style="font-size: 150%;">
$$p(X|Y=y_i) = \frac{1}{\sigma_i\sqrt{2\pi}}\, e^{-\frac{(x - \mu_i)^2}{2 \sigma_i^2}}$$
</span>

The mean ($\mu$) and standard deviation ($\sigma$) for the two classes is:
* $y_1$: $\mu_1 = -2$ and $\sigma_1 = 1$.
* __$y_2$__: $\mu_1 = 2$ and $\sigma_1 = 1$

Furthermore, we assue that the two classes are equiprobable (the have the same prior probability) and assume the classes to be equiprobable, meaning: $$P(Y=y_1) = P(Y=y_2) = 0.5$$

We can express this by implementing the following function that defines the pdf.

In [ ]:
def normal_pdf(x: Union[float, np.ndarray], mu: float, sigma: float) -> Union[float, np.ndarray]:
    """
    Calculate the probability density function of a normal (Gaussian) distribution.

    This function computes the likelihood of an observation `x` given the mean `mu`
    and standard deviation `sigma` of a normal distribution, often used as the 
    class-conditional probability in Naive Bayes classification.

    :param x: The data point(s) for which the PDF is evaluated. Can be a single float or a numpy array.
    :param mu: The mean (μ) of the normal distribution.
    :param sigma: The standard deviation (σ) of the normal distribution.
    :return: The probability density value(s) for `x` under the specified normal distribution.
    """
    # Calculate the constant factor of the Gaussian formula
    coefficient = 1 / (np.sqrt(2 * np.pi) * sigma)
    
    # Calculate the exponential factor in the Gaussian formula
    exponent = np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))
    
    # Return the product of the coefficient and exponent for the PDF
    return coefficient * exponent

__Exercise 1__: Using the previous function plot the likelihood of each class.

__Tasks__:
* Define the the class priors $P(Y=y_1)$ and $P(Y=y_2)$.
* Define the mean and standard devation of both classes.
* Calculate the pdfs for both classes using an appropriate range for $x$.
* Plot the two pdfs into a single plot.

In [ ]:
# define class prioirs
prior_y1 =
prior_y2 =

# define mean and standard deviation 
mean_y1 =
std_y1 =
mean_y2 =
std_y2 =

# define the range for x
dx = 0.1
x = np.arange(-7, 7, dx)

# calculate pdfs for both classes
pdf_x1 =
pdf_x2 = 

# Plot
plt.figure(figsize=(10, 5))


# add title and labels
plt.xlabel('Feature values')
plt.ylabel('Density')
plt.title('Likelihood')
plt.legend();

__Exercise 2__: Calculate the __marginal probability__ (or evidence) over the defined feature vector and plot the result.

In [ ]:
# calculate the marginal probabilty
p_x = 

# plot the marginal probability
plt.figure(figsize=(10, 5))


# add title and labels
plt.title("Marginal Probability")
plt.xlabel('Feature values')
plt.ylabel('Density')
plt.legend();

__Exercise 3__: Calculate the __a posteriori class probability__ for each class according to the Bayes' Rule and plot them both into the same plot.

In [ ]:
# calculate a posteriori probability
post_prob_y1 = 
post_prob_y2 = 

# plot both a posteriori probability
plt.figure(figsize=(10, 5))

# add title and labels
plt.title("Posterior Probability") 
plt.xlabel('Feature values')
plt.ylabel('Probability')
plt.legend();

__Exercise 4__: 
Now that we have calculated the a posteriori probability, we can use it to calculate the probabilty of new data points $X$ belonig to either class $y_1$ or $y_2$.

__Tasks__:
* Calculate the class probability for the points $x_1=-1$ and $x_2=0$ for both classes.
* Plot the points as vertical lines into the plot from __Exercise 3__.

In [ ]:

# calculate the probabilty of x_1 belonging to either class y_1 or y_2 and print the result
print('x = -1')
print('Probability of Class 1:', )
print('Probability of Class 2:', )


# calculate the probabilty of x_2 belonging to either class y_1 or y_2 and print the result
print('\nx = 0')
print('Probability of Class 1:', )
print('Probability of Class 2:', )

In [ ]:
# Plot the figure with the decision boundaries
plt.figure(figsize=(10, 5))

# add title and labels
plt.title("Posterior Probability") 
plt.xlabel('Feature values')
plt.ylabel('Probability')
plt.legend();

#### 3.4 The Discriminant Function

In statistical machine learning, a discriminant function is a formula that helps us decide which class a given data point most likely belongs to. Specifically, it provides a numerical score for each class based on certain characteristics of the data, known as features. __For classification, we calculate the discriminant function for each possible class and then assign the data point to the class with the highest score__.

When using Bayes' Rule, the discriminant function is the posterior probability:

$$ g_i(x) = P(Y=y_i|X) = \frac{p(X|Y=y_i)P(Y=y_i)}{P(X)}$$

Given that the final decision on which class is chosen is made based on which of the discriminant function has the highest value, the discriminant function when using Bayes' Rule can be simplified to:

$$ g_i(x) = p(X|Y=y_i)P(Y=y_i) $$

This is due to the fact that the denominator ($P(X)$) is the same for all classes.

To avoid the multiplication of small numbers ofen times the logarithm is used. Thus, the discriminat function becomes:

$$ g_i(x) = \log(P(Y=y_i|X)) = \log(p(X|Y=y_i)) + \log(P(Y=y_i))$$

__Exercise 5__: Define the discriminant functions for this problem. You can use the definition without the log.

In [ ]:
g1 = 
g2 = 

#### 3.5 Bayes Error - Calculating the Classification Error

The __Bayes error__ represents the minimum achievable classification error for a given problem, assuming we use the optimal decision rule. In other words, it is the error rate of the Bayes classifier, which chooses the class $Y$ that maximizes the posterior probability $P(Y|X)$. Mathematically, assuming that the distribution of $P(X)$ is either uniform or we do not have any prior knowledge about, we can estimate the Bayes error as:

$$P_e = 1 - \int_{-\infty}^{\infty}\max_i(g_i(x)) dx$$

where:
* $max_i(g_i(x))$ is the discriminant function that assigns the class with the highest posterior probability for each value of $x$.

#### 3.6 Simplified Bayes Error with Symmetric Error Regions

When the class distributions are __symmetric__, the Bayes error can be alternatively calculated by focusing only on one side of the decision boundary and doubling the result. For example, if we have two normal distributions with equal variances but different means, we can denote the decision boundary (or separation point) as $x_{sep}$, where the class-conditional probabilities are equal:

$$p(X|Y=y_1) = p(X|Y=y_2) at X=x_{sep}$$

In this case, the Bayes error can be computed as:

$$P_e = 2\int_{x_{sep}}^{+\infty} p(X|Y=y_1)P(Y=y_1) dX = 2\int_{-\infty}^{x_{sep}} p(X|Y=y_2)P(Y=y_2) dX$$

where:

* $\int_{x_{sep}}^{+\infty} p(X|Y=y_1)P(Y=y_1) dX$: is the __false negatives__.
* $\int_{-\infty}^{x_{sep}} p(X|Y=y_2)P(Y=y_2) dX$: is the __false positives__.


<div style="text-align:center;">
    <img src="https://raw.githubusercontent.com/hgamboa/nova-aaeb/refs/heads/main/images/Bayes%20Error.png" width="900">
</div>

__Exercise 6__: What is the error of this classification problem? Let's calculate the error using 3 different approaches:

__Exercise 6.1__: Derive the error formula by integrating the discriminative functions of __Exercise 5__ using the __non-simplified__ version of the Bayes Error $P_e$.

In [ ]:

# calculate bayes error (error rate)
bayes_error =

# print the result
print('Error:', bayes_error)

__Exercise 6.2__: As our problem has symmetrical error regions, calculate the error using the __simplified__ formula of the Bayes Error and compare it with the error calculated in __Exercise 6.1__.

For simplicity, you can approximate the error using the following formula:

$$cdf = \int pdf$$

$$cdf = \frac12\left[1 + \operatorname{erf}\left( \frac{x_{sep}-\mu}{\sigma\sqrt{2}}\right)\right] P(Y) $$

In [ ]:
from scipy.special import erf

# define decision boundary
x_sep = 

# calculate the error using the simplified formula
error_a = 

print('Error:', error_a)

__Exercise 6.3__: Now, we are going to use the Naive Bayes Classifier implementation from scikit-learn. We will train it and then calculate the error through the functions provided by scikit-learn.

__Tasks__:

* Random generate data from the two classes.
* Create the label vectors for each class. You can use class labels 1 for class $y_1$ and 0 for class $y_2$.
* Plot the genrated $X$ data, to verify that the data is correct.
* Split the data into train and test sets.
* Use sklearn's implementation of the Naive Bayes classifier to fit the train data and predict the test data.
* Compare the obtained error with the __Exercise 6.1__ and __Exercise 6.2__.

__Hints__:
* [numpy.random.normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html) can be used to draw samples from a gaussian distribution.
* For plotting the generated data you can use [matplotlib.pyplot.hist](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html).
* [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) is the Naive Bayes classifier impementation of scikit-learn.
* Option 1: You can calculate the error by: 
$$ E = \frac{\sum_{i=1}^{N} \mathbb{1}(y_i \neq \hat{y}_i)}{N}$$
where $N$ is the total number of samples and $\mathbb{1}(y_i \neq \hat{y}_i)$ is an indicator function that equals 1 if $y_i \neq \hat{y}_i$ (indicating a misclassification) and 0 otherwise.
* Option 2: You can calculate the error by:
$$E = 1 - accuracy$$

In [ ]:
# Data generation

# define the number of points to be generated
n_points = 500000

# draw the samples for x (for each class) from the gaussian distribution
x1 = np.random.normal(loc=mean_y1, scale=std_y1, size=n_points)
x2 = np.random.normal(loc=mean_y2, scale=std_y2, size=n_points)

# create feature matrix and labels
X = np.concatenate((x1, x2))
y = np.concatenate((np.zeros(n_points), np.ones(n_points)))

# plot the data 
plt.figure()
plt.hist(X,100)
plt.show()

In [ ]:
# Split, fit and predict

# reshape the data for training
X = X.reshape(-1, 1)

# split data

# train model

# predict test data


In [ ]:
# Error calculation



__Exercise 6.3.1__: Observe and compare the learned parameters (mean and standard deviation) with the real values.